In [13]:
from collections import defaultdict
import heapq

class InformedGraph:
    def __init__(self):
        self.graph = defaultdict(list)  # graph[node] = [(neighbor, cost), ...]
        self.heuristics = {}  # heuristics[node] = h(n)

    # Add undirected edge
    def add_edge(self, u, v, cost=1):
        self.graph[u].append((v, cost))
        self.graph[v].append((u, cost))

    # Set heuristic
    def set_heuristic(self, node, value):
        self.heuristics[node] = value

    # ---------------- Best First Search ----------------
    def best_first_search(self, start, goal):
        visited = set()
        pq = [(self.heuristics.get(start, float('inf')), start, [start])]

        while pq:
            _, current, path = heapq.heappop(pq)

            if current == goal:
                return path

            if current not in visited:
                visited.add(current)
                for neighbor, _ in self.graph.get(current, []):
                    if neighbor not in visited:
                        new_path = path + [neighbor]
                        heapq.heappush(pq, (self.heuristics.get(neighbor, float('inf')), neighbor, new_path))
        return None

    # ---------------- Beam Search ----------------
    def beam_search(self, start, goal, beam_width):
        beam = [([start], self.heuristics.get(start, float('inf')))]
        visited = {start}

        while beam:
            successors = []
            
            for path, _ in beam:
                current_node = path[-1]

                if current_node == goal:
                    return path

                for neighbor, _ in self.graph.get(current_node, []):
                    if neighbor not in visited:
                        visited.add(neighbor)
                        new_path = path + [neighbor]
                        score = self.heuristics.get(neighbor, float('inf'))
                        successors.append((new_path, score))
            
            if not successors:
                break
            
            successors.sort(key=lambda x: x[1])
            beam = successors[:beam_width]

        return None

    # ---------------- A* Search----------------
    def a_star_search(self, start, goal):
    
        g_score_start = 0
        h_score_start = self.heuristics.get(start, float('inf'))
        f_score_start = g_score_start + h_score_start
        
        pq = [(f_score_start, g_score_start, start, [start])]
        
        g_scores = defaultdict(lambda: float('inf'))
        g_scores[start] = 0

        while pq:
            _, g_score, current, path = heapq.heappop(pq)

            if current == goal:
                return path

            if g_score > g_scores[current]:
                continue

            for neighbor, cost in self.graph.get(current, []):
                new_g_score = g_score + cost

                if new_g_score < g_scores[neighbor]:
                    g_scores[neighbor] = new_g_score
                    h_score = self.heuristics.get(neighbor, float('inf'))
                    f_score = new_g_score + h_score
                    new_path = path + [neighbor]
                    heapq.heappush(pq, (f_score, new_g_score, neighbor, new_path))
        
        return None 


# ---------------- Main Program----------------
if __name__ == "__main__":
    g = InformedGraph()

    # Take graph input
    n = int(input("Enter number of edges: "))
    print("Enter edges in format: node1 node2 cost")
    for _ in range(n):
        u, v, c = input().split()
        g.add_edge(u, v, int(c))

    # Take heuristics
    h = int(input("Enter number of heuristic values: "))
    print("Enter heuristic in format: node value")
    for _ in range(h):
        node, val = input().split()
        g.set_heuristic(node, int(val))

    # Start and goal
    start = input("Enter start node: ")
    goal = input("Enter goal node: ")

    # Beam width for Beam Search
    beam_width = int(input("Enter beam width for Beam Search: "))

    print("\n" + "="*30)
    print("--- Running Best First Search ---")
    path_bfs = g.best_first_search(start, goal)
    if path_bfs:
        print("Best First Search Path:", " → ".join(path_bfs))
    else:
        print("Goal not reachable with Best First Search")

    print("\n--- Running Beam Search ---")
    path_beam = g.beam_search(start, goal, beam_width)
    if path_beam:
        print(f"Beam Search Path (width={beam_width}):", " → ".join(path_beam))
    else:
        print("Goal not reachable with Beam Search")

    print("\n--- Running A* Search ---")
    path_astar = g.a_star_search(start, goal)
    if path_astar:
        print("A* Search Path:", " → ".join(path_astar))
    else:
        print("Goal not reachable with A* Search")
    print("="*30)

Enter number of edges:  6


Enter edges in format: node1 node2 cost


 A B 4
 A C 3
 B D 2
 B D 6
 C F 3
 C G 5
Enter number of heuristic values:  7


Enter heuristic in format: node value


 A 15
 B 12
 C 10
 D 8
 E 7
 F 9
 G 0
Enter start node:  A
Enter goal node:  G
Enter beam width for Beam Search:  1



--- Running Best First Search ---
Best First Search Path: A → C → G

--- Running Beam Search ---
Beam Search Path (width=1): A → C → G

--- Running A* Search ---
A* Search Path: A → C → G
